## Check List 3 - SceneBuilder, GeometryHandle
* **3.1 SceneBuilder**  
  - reset_reference_coord, detect_items, create_gscene, add_robot_geometries, add_poles, detect_and_register
* **3.2 GeometryHandle**  
  - create_safe
    - BOX, CYLINDER, PLANE, CAPSULE, SPHERE, ARROW
  - show_pose, show_motion

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 3.1. SceneBuilder

##### Create CombinedRobot instance - you need it to initailize GeometryScene

In [2]:
from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.0.63"
PANDA_HOST_IP = "192.168.0.172" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.0.13" ## The robot has it's own IP

combined_robot = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))], 
    connection_list=[False, False])

connection command:
indy0: False
panda1: False


##### initialize ArucoStereo detector

In [3]:
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

stereo = ArucoStereo(aruco_map=get_aruco_map(), 
                     camera_list=[Kinect(), RealSense()])

stereo.initialize()

time.sleep(1) # Let the camera have some time to get stable
stereo.calibrate()
print("Detector initialized")

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming
Detector initialized


##### Create SceneBuilder instance

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
scene_builder = SceneBuilder.instance(detector=stereo)

##### reset_reference_coord()
* Set the name of coordinate reference object - here, you need "floor" as reference

In [5]:
scene_builder.reset_reference_coord(ref_name="floor")

##### detect_items()
* Detect items in the field of view. Here, we detect robot positions and update it to the combined robot

In [6]:
xyz_rpy_robots = scene_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
combined_robot.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)

##### create_gscene()
* Create a GeometryScene instance

In [7]:
gscene = scene_builder.create_gscene(combined_robot)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


##### detect_and_register()
* Detect items in the field of view and register them to the GeometryScene
* They will appear in the RVIZ

In [8]:
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

Please create a subscriber to the marker


## Vive
#### Prepare VR
1. Start vrmonitor
2. Turn on trackers - push until led blinks or change to green - Trackers will be displayed green on the SteamVR
  - Turn on the reference tracker first

In [9]:
from pkg.utils.shared_function import clear_channels_on
from pkg.detector.vive.shared_vive import SharedTrackers, SERVER_ID

In [10]:
# clear_channels_on(SERVER_ID)
track = SharedTrackers()

In [11]:
track.init_vive()

In [12]:
vive_ref = gscene.NAME_DICT["vive_ref"]
tf_base_ref = vive_ref.get_tf(combined_robot.home_pose)
track.set_reference(ref_name="tracker_1", tf_base_ref=tf_base_ref)

In [14]:
for _ in range(100):
    try:
        pose_dict = track.get_all_pose()
        for tname, T in pose_dict.items():
            gscene.add_highlight_axis("hl", tname, T=np.array(T))
        time.sleep(0.1)
    except Exception as e:
        print(e)

In [16]:
from pkg.utils.rotation_utils import *

In [17]:
T1 = SE3(Rot_rpy([0, 0, np.pi]), (0.1, 0.05,0.0))

In [18]:
T2 = matmul_series(SE3(Rot_axis(3,np.pi), (0,0,0)), T1, )

In [33]:
gscene.add_highlight_axis("hl", "t2", T=T2)

In [19]:
np.round(T2[:3,3], 3)

array([-0.1 , -0.05,  0.  ], dtype=float32)

In [20]:
np.round(Rot2rpy(T2[:3,:3])/np.pi, 3)

array([ 0., -0., -0.])